In [4]:
# combine sentiment by stanford
import pandas as pd
import numpy as np
df = pd.read_csv('sentiment_tweet.csv', quotechar='"')
del df['text']
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['user']

import ast

def getSent(row):
    sentiment = ast.literal_eval(row['sentiment'])['sentiment']
    stanford = row['stanford']
    if stanford == 1:
        stanford = 1
    if stanford == -1:
        stanford = -1
    if stanford == 2:
        stanford = 1
    if stanford == -2:
        stanford = -1
    if sentiment == None:
        return stanford
    else:
        if sentiment['basic'] == 'Bullish':
            return 2
        if sentiment['basic'] == 'Bearish':
            return -2
        return stanford
    
df['sentiment'] = df.apply(getSent, axis=1)
del df['stanford']
df.head()
df['sentiment'].value_counts()

-1    81348
 2    63777
 0    41912
-2    25351
 1    10588
Name: sentiment, dtype: int64

In [6]:
# aggregate by date
def getIndex(arr):
    bull = 0
    bear = 0
    for i in arr:
        if i > 0:
            bull += i
        if i < 0:
            bear += -i
#     return bull-bear
    if bull == bear:
        return 0
    elif bull > bear:
        return ((2*bull)/(bull+bear) - 1) / len(arr)
    else:
        return (1 - (2*bear)/(bull+bear)) / len(arr)

data = df.groupby('date').agg({'sentiment': getIndex})['sentiment']
data.to_csv('aggregate_tweet.csv')

In [7]:
#add time series to sentiment
import datetime as dt
df = pd.read_csv("./AAP_data.csv", usecols=[0,1,2,3,4,5])
df['date'] = df['date'].map(lambda x: dt.datetime.strptime(x,"%Y-%m-%d").strftime('%Y.%m.%d'))

data = pd.read_csv('./aggregate_tweet.csv', header=None, names = ['date', 'sentiment'])
data = data.set_index('date')
df = df.set_index('date')
result = pd.concat([data, df], axis=1, join='inner')
result.to_csv('./merged_result.csv')
result = pd.read_csv('./merged_result.csv', usecols=[1,5,6])
result.head()

result['output'] = result.close.shift(-1)    

result.dropna(inplace=True)
result.reset_index(drop=True,inplace=True)
try:
    result = result.drop(columns=['date'])
except:
    pass

In [8]:
#preprocess dataframe
up = 1
down = -1
def setLabel(row):
    yesterday = row['close']
    today = row['output']
#     return today
    return today - yesterday
    if yesterday < today:
        return up
    if today < yesterday:
        return down
    return 0

result['output'] = result.apply(setLabel, axis=1)

def norm(a):
    b = a - a.min()
    return b / b.max()

result['sentiment'] = norm(result['sentiment'])
result['output'] = norm(result['output'])
result['close'] /= result['close'].max()
result['volume'] /= result['volume'].max()
result.to_csv('./ML_cols.csv', encoding='utf-8')
result.head()

,sentiment,close,volume,output
0,0.000000,0.630564,0.301822,0.303628
1,0.847330,0.626455,0.181174,0.271155
2,0.779408,0.617696,0.141735,0.369706
3,0.852223,0.623049,0.153678,0.201238
4,0.864224,0.604279,0.232292,0.299457


In [171]:
#fit model

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_dataset1(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = []
        for j in range(look_back):
            a += dataset[i+j, :-1].tolist()
        dataX.append(a)
        dataY.append(dataset[i + look_back, -1])
    return numpy.array(dataX), numpy.array(dataY)

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(look_back-1, len(dataset)):
        a = []
        for j in range(look_back):
            a += dataset[i-j, :-1].tolist()
        dataX.append(a)
        dataY.append(dataset[i, -1])
    return numpy.array(dataX), numpy.array(dataY)

numpy.random.seed(7)

dataframe = pd.read_csv('./ML_cols.csv', usecols=[1,2,3,4])
dataset = dataframe.values
dataset = dataset.astype('float32')
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# trainX = x_scaler.fit_transform(trainX)
# trainY  = y_scaler.fit_transform(trainY.reshape(-1,1))
# testX = x_scaler.transform(testX)
# testY = y_scaler.transform(testY.reshape(-1,1))

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

model = Sequential()
model.add(LSTM(input_shape=(3 * look_back, trainX.shape[2]), return_sequences=True, units=50))
model.add(Dropout(0.5))
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=30, batch_size=1, verbose=2)


Epoch 1/30
 - 6s - loss: 0.0151
Epoch 2/30
 - 2s - loss: 0.0113
Epoch 3/30
 - 2s - loss: 0.0126
Epoch 4/30
 - 2s - loss: 0.0117
Epoch 5/30
 - 2s - loss: 0.0128
Epoch 6/30
 - 2s - loss: 0.0095
Epoch 7/30
 - 2s - loss: 0.0098
Epoch 8/30
 - 2s - loss: 0.0089
Epoch 9/30
 - 4s - loss: 0.0115
Epoch 10/30
 - 4s - loss: 0.0100
Epoch 11/30
 - 3s - loss: 0.0100
Epoch 12/30
 - 3s - loss: 0.0089
Epoch 13/30
 - 2s - loss: 0.0091
Epoch 14/30
 - 2s - loss: 0.0096
Epoch 15/30
 - 2s - loss: 0.0097
Epoch 16/30
 - 2s - loss: 0.0091
Epoch 17/30
 - 2s - loss: 0.0105
Epoch 18/30
 - 2s - loss: 0.0093
Epoch 19/30
 - 3s - loss: 0.0091
Epoch 20/30
 - 2s - loss: 0.0087
Epoch 21/30
 - 2s - loss: 0.0093
Epoch 22/30
 - 2s - loss: 0.0089
Epoch 23/30
 - 2s - loss: 0.0092
Epoch 24/30
 - 2s - loss: 0.0091
Epoch 25/30
 - 2s - loss: 0.0092
Epoch 26/30
 - 2s - loss: 0.0087
Epoch 27/30
 - 2s - loss: 0.0091
Epoch 28/30
 - 2s - loss: 0.0081
Epoch 29/30
 - 2s - loss: 0.0090
Epoch 30/30
 - 3s - loss: 0.0091


In [178]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainCorrect = trainY
testCorrect = testY
# trainPredict = y_scaler.inverse_transform(trainPredict)
# trainCorrect = y_scaler.inverse_transform(trainY)
# testPredict = y_scaler.inverse_transform(testPredict)
# testCorrect = y_scaler.inverse_transform(testY)
import math

def sign(a):
    if a>0:
        return 1
    if a<0:
        return -1
    return 0

def acc(a, b):
    cnt = 0
    for i in range(len(a)):
        print(a[i], b[i])
        if sign(a[i]) == sign(b[i]):
            cnt += 1
    return
    return cnt / len(a)

trainScore = math.sqrt(mean_squared_error(trainCorrect[:], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
print()
print("acc: ", acc((trainCorrect[:]), (trainPredict[:,0])))
print()
print()
print()
testScore = math.sqrt(mean_squared_error(testCorrect[:], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
print()
print("acc: ", acc((testCorrect[:]), (testPredict[:,0])))


Train Score: 0.09 RMSE

0.27115458 0.31478444
0.3697064 0.31914535
0.20123753 0.31863683
0.2994565 0.31953016
0.4807885 0.31990346
0.34138933 0.31983858
0.36896887 0.3206736
0.42941162 0.3206293
0.08223556 0.32006925
0.33156705 0.32047957
0.42941162 0.32074523
0.4044796 0.32023084
0.16307929 0.31983626
0.27339363 0.32032788
0.39994478 0.31973302
0.28661227 0.31935194
0.31872284 0.31965947
0.18272384 0.31923604
0.3262796 0.32039595
0.3984357 0.32016012
0.30172202 0.32030463
0.24883604 0.32087785
0.27868107 0.3206346
0.39881012 0.32007647
0.3855877 0.31951147
0.36594692 0.3197288
0.5348091 0.31983155
0.45170367 0.32067904
0.26092383 0.32115537
0.37500897 0.32035923
0.34441507 0.31994468
0.31456625 0.31996435
0.37916937 0.31998777
0.31230074 0.31967854
0.35725552 0.3196913
0.37727827 0.319755
0.34138933 0.3207694
0.39692283 0.3204643
0.39541376 0.32011
0.2684806 0.32046044
0.29870006 0.31959242
0.3610339 0.31966084
0.27679 0.32003763
0.3636814 0.3200564
0.1880113 0.32047153
0.63152283 0.3